In [2]:
import cv2 
import pandas as pd
import numpy as np
import math

In [3]:
class FaceToDF:
    def __init__(self):
        self.detector = cv2.FaceDetectorYN.create(
            model='C:/Users/Stepan/Documents/ClassMood/yunet_headpose/face_detection_yunet_2023mar.onnx', #insert path to the face_detection_yunet_2023mar.onnx file
            config='',
            input_size=(67, 69),
            score_threshold=0.8,
            nms_threshold=0.3,
            top_k=5000
        )
        
        self.matrix = np.zeros((3, 3))
        self.dist_coeffs = np.zeros((1, 5))
    
        if cv2.cuda.getCudaEnabledDeviceCount() > 0:
            self.detector.setBackend(cv2.dnn.DNN_BACKEND_CUDA)
            self.detector.setTarget(cv2.dnn.DNN_TARGET_CUDA)
    
    def calibrate_matrix(self, chb_size, images=None, video=None):
        chessboard_size = (chb_size[0] - 1, chb_size[1] - 1)
        objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), dtype=np.float32)
        objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
        objpoints, imgpoints = [], []
        calibration_data = []

        if images:
            if len(images) < 10:
                raise Exception('Пожалуйста, введите не менее 10 фотографий')
            calibration_data = images
            calibration_mode = 'img'
        else:
            cap = cv2.VideoCapture(video)
            if (cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)) < 2:
                raise Exception('Пожалуйста, введите видео длинной более двух секунд')
            
            frame_skip = cap.get(cv2.CAP_PROP_FRAME_COUNT) // 5
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                if frame_count % frame_skip == 0:
                    calibration_data.append(frame)
                frame_count += 1
            calibration_mode = 'vid'

        for path in calibration_data:
            if calibration_mode == 'img':
                img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
            else:
                img = cv2.cvtColor(path, cv2.COLOR_BGR2GRAY)
            found, corners = cv2.findChessboardCorners(img, chessboard_size, None)

            if found:
                objpoints.append(objp)

                corners_refined = cv2.cornerSubPix(img, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
                imgpoints.append(corners_refined)
        h, w = img.shape[:2]

        ret, camera_matrix, self.dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (w, h), None, None)
        self.matrix = cv2.getOptimalNewCameraMatrix(camera_matrix, self.dist_coeffs, (w, h), 1, (w, h))[0]
        
    def head_pose_img(self, path=None, frame=None):
        if path:
            image = cv2.imread(path)
        else:
            image = frame
        self.detector.setInputSize(image.shape[1::-1])
        faces = self.detector.detect(image)[1]
        
        if faces is None:
            return pd.DataFrame()
        
        columns = [
            'face_id', 'x', 'y', 'width', 'height',
            'le_x', 'le_y',
            're_x', 're_y',
            'n_x', 'n_y',
            'ml_x', 'ml_y',
            'mr_x', 'mr_y',
            'confidence',
            'yaw', 'pitch', 'roll'
        ]
        model_3d_points = np.array([
            [-20.0, -10.0, -40.0],   # левый глаз
            [ 20.0, -10.0, -40.0],   # правый глаз
            [  0.0,  0.0, -35.0],    # нос (немного ближе)
            [-15.0, 10.0, -42.0],    # левый угол рта (глубже)
            [ 15.0, 10.0, -42.0]     # правый угол рта (глубже)
        ], dtype=np.float32)
        data = []

        for i, face in enumerate(faces):
            success, rvec, tvec = cv2.solvePnP(model_3d_points, np.array(face[4:14]).reshape(-1, 1, 2), self.matrix, self.dist_coeffs, flags=cv2.SOLVEPNP_EPNP)
            R = cv2.Rodrigues(rvec)[0]
            sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
            if sy > 0.000001:
                yaw = math.atan2(R[1, 0], R[0, 0])
                pitch = math.atan2(-R[2, 0], sy)
                roll = math.atan2(R[2, 1], R[2, 2])
            else:
                yaw = math.atan2(-R[1, 2], R[1, 1])
                pitch = math.atan2(-R[2, 0], sy)
                roll = 0.0
            yaw = round(math.degrees(yaw), 5)
            pitch = round(math.degrees(pitch), 5)
            roll = round(math.degrees(roll), 5)
            row = [i + 1] + face.tolist() + [yaw, pitch, roll]
            data.append(row)
        
        df = pd.DataFrame(data, columns=columns)
        return df

    def head_pose_vid(self, path):
        cap = cv2.VideoCapture(path)
        frame_count = 0
        fps = cap.get(cv2.CAP_PROP_FPS)
        main_df = pd.DataFrame(columns=['frame', 'sec', 'face_id', 'x', 'y', 'width', 'height', 'yaw', 'pitch', 'roll'])

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break
            
            if frame_count % 5 == 0:
                temp = self.head_pose_img(frame=frame)
                new_row = {
                    'frame': frame_count, 'sec': round(frame_count / fps, 5), 'face_id': temp['face_id'].tolist(), 
                    'x': temp['x'].tolist(), 'y': temp['y'].tolist(), 
                    'width': temp['width'].tolist(), 'height': temp['height'].tolist(),
                    'yaw': temp['yaw'].tolist(), 'pitch': temp['pitch'].tolist(), 'roll': temp['roll'].tolist()
                }
                main_df = pd.concat([main_df, pd.DataFrame([new_row])], ignore_index=True)
            frame_count += 1
        
        return main_df
            

            
            


In [ ]:
face_to_dataframe = FaceToDF()
face_to_dataframe.calibrate_matrix((8, 8), images=[f'C:/Users/Stepan/Documents/ClassMood/yunet_headpose/images/calib_images/{i}.jpg' for i in range(1, 11)])

In [ ]:

face_to_dataframe.head_pose_img('C:/Users/Stepan/Documents/ClassMood/yunet_headpose/images/test_images/img3.jpg')

In [4]:
face_to_dataframe = FaceToDF()
face_to_dataframe.calibrate_matrix((8, 8), video='C:/Users/Stepan/Documents/ClassMood/yunet_headpose/videos/calib/vid1.MOV')

In [47]:
result = face_to_dataframe.head_pose_vid('C:/Users/Stepan/Documents/ClassMood/yunet_headpose/videos/vid2.MOV')

C:\Users\Stepan\AppData\Local\Temp\ipykernel_14708\351944604.py:136: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  main_df = pd.concat([main_df, pd.DataFrame([new_row])], ignore_index=True)


In [43]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [51]:
result

,frame,sec,face_id,x,y,width,height,yaw,pitch,roll
0,0,0.00000,[1],[208.07907104492188],[391.52471923828125],[275.532470703125],[448.6540832519531],[6.66542],[56.35328],[4.41916]
1,5,0.16667,[1],[208.5932159423828],[391.70697021484375],[274.6446228027344],[444.4373779296875],[6.34445],[56.35643],[4.59029]
2,10,0.33333,[1],[207.8606414794922],[392.1642150878906],[275.6554260253906],[446.43487548828125],[2.34761],[-58.99178],[10.71585]
3,15,0.50000,[1],[207.58338928222656],[390.998046875],[276.0649108886719],[447.4754638671875],[1.95311],[-59.19391],[10.96036]
4,20,0.66667,[1],[208.8228302001953],[390.3823547363281],[275.2547302246094],[446.50152587890625],[5.25943],[56.01583],[4.52262]
5,25,0.83333,[1],[215.20306396484375],[392.05328369140625],[273.41107177734375],[441.9289245605469],[4.5729],[55.90116],[4.04958]
6,30,1.00000,[1],[216.32322692871094],[390.6324462890625],[273.5898132324219],[440.8379821777344],[3.00754],[57.30131],[5.20965]
7,35,1.16667,[1],[220.81407165527344],[385.0318908691406],[279.1188049316406],[448.70416259765625],[3.86268],[56.85098],[5.44974]
8,40,1.33333,[1],[234.93519592285156],[388.22467041015625],[269.5653381347656],[440.0795593261719],[3.50022],[59.85246],[5.34796]
9,45,1.50000,[1],[233.3777313232422],[384.21612548828125],[275.3501892089844],[444.8654479980469],[3.21954],[61.45095],[6.89943]


In [39]:
model_3d_points = np.array([
    [-20.0, -8.0, -40.0],   # левый глаз
    [ 20.0, -10.0, -40.0],   # правый глаз
    [  0.0,  0.0, -38.0],    # нос (немного ближе)
    [-15.0, 10.0, -42.0],    # левый угол рта (глубже)
    [ 15.0, 10.0, -42.0]     # правый угол рта (глубже)
], dtype=np.float32)
image = cv2.imread('C:/Users/Stepan/Documents/ClassMood/yunet_headpose/images/full_face_images/img11.jpg')
face_to_dataframe.detector.setInputSize(image.shape[1::-1])
faces = face_to_dataframe.detector.detect(image)[1]
face = faces[0]
success, rvec, tvec, inliers = cv2.solvePnPRansac(model_3d_points, np.array(face[4:14]).reshape(-1, 1, 2), face_to_dataframe.matrix, face_to_dataframe.dist_coeffs, flags=cv2.SOLVEPNP_EPNP, reprojectionError=2.0, iterationsCount=1)
projected_points, _ = cv2.projectPoints(model_3d_points, rvec, tvec, face_to_dataframe.matrix, face_to_dataframe.dist_coeffs)

for p in projected_points:
    x, y = int(p[0][0]), int(p[0][1])
    cv2.circle(image, (x, y), 3, (0, 255, 0), -1)
cv2.imwrite('C:/Users/Stepan/Documents/ClassMood/yunet_headpose/images/images_with_landmarks/image3.png', image)

True

In [ ]:
face_to_dataframe = FaceToDF()


def draw_landmarks_on_image(image, landmarks, labels=None, color=(0, 0, 255), radius=1, thickness=-1, font_scale=0.4, font_thickness=1):
    output_image = image.copy()
    for row in landmarks:
        for i, (x, y) in enumerate(row):
            cx, cy = int(x), int(y)
        
            cv2.circle(output_image, (cx, cy), radius, color, thickness)
        
            if labels and i < len(labels):
                cv2.putText(
                    output_image,
                    str(labels[i]),
                    (cx + 5, cy - 5),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale,
                    (255, 0, 0),
                    font_thickness
                )

        cv2.putText(
                output_image,
                str(round(((row[0][0] + row[1][0] + row[3][0] + row[4][0]) / 4 - row[2][0]) / (row[0][0] - row[1][0] + row[3][0] - row[4][0]), 6)),
                (cx + 5, cy - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale,
                (0, 255, 255),
                font_thickness
            )

    return output_image


def processing_landmarks(imgpath):
    landmarks = []
    raw_data = face_to_dataframe.head_pose_img(imgpath)
    for i in range(len(raw_data)):
        landmarks.append(raw_data[i][4:-1].reshape(-1, 2))

    image_with_landmarks = draw_landmarks_on_image(
        image=cv2.imread(imgpath),
        landmarks=landmarks,
        labels=None, #['LE', 'RE', 'N', 'LM', 'RM']
        color=(0, 0, 255),
        radius=5
    )

    output_path = 'image_with_landmarks.jpg' #using imgpath as the output_path overrides the input image. to avoid that use something like 'image_with_landmarks.jpg'
    cv2.imwrite(output_path, image_with_landmarks)

processing_landmarks('C:/Users/Stepan/Documents/ClassMood/yunet_headpose/test_img30.jpg') #use to draw landmarks on images 